# Repetition Code Simulations

## IMPORTS

- python version: 3.12.11
- sinter version: 1.15.0
- numpy version: 2.2.6

In [2]:
import sys
! {sys.executable} --version


Python 3.12.11


In [3]:
import stim
import numpy as np
import pymatching

import networkx as nx
import matplotlib.pyplot as plt
from itertools import chain

import sinter
from typing import List


import pandas as pd
from scipy.optimize import fsolve

from decimal import Decimal

import random

from scipy.optimize import fsolve


In [4]:
import mwpf
from mwpf.sinter_decoders import SinterMWPFDecoder

from mwpf.ref_circuit import *
from mwpf.heralded_dem import *

In [5]:
print(np.__version__)

2.2.6


In [6]:
print(sinter.__version__)

1.15.0


In [7]:
def one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len: int, heralded_erasure_err):

    circuit = stim.Circuit()
    
    circuit. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    for q in erase_qubits:
        circuit. append_operation ("DETECTOR", [stim.target_rec(-(q + 1))])

    return circuit

# One Heralded Error Per Cycle

In [8]:
def one_det_get_circuit_and_dem(rounds, distance, bit_flip_rate_per_round, heralded_erasure_err, misassign_err, case = 1):

    # initialize circuit info
    meas_len = distance - 1
    all_len = ((2 * distance) - 1)
    erase_len = all_len
    if case == 1:
        erase_factor = 4
    elif case == 2:
        erase_factor = 5
    
    qubits = list(range(all_len))
    meas_qubits = list(qubits [1::2])
    data_qubits = list(qubits[0::2])
    erase_qubits_0 = list(range(0,all_len*3))
    erase_qubits = list(range(0,all_len*erase_factor))

    cnot1 = qubits [:-1:1]
    cnot2 = qubits [:0:-1]
    

    
    

    # Build circuit.

    c_i = stim.Circuit()
    c_i. append_operation ("R", qubits)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    ### CNOTS
    c_i. append_operation ("CNOT", cnot1)
    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    c_i. append_operation ("CNOT", cnot2)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits_0, qubits, erase_len, heralded_erasure_err)
    if case == 2:   
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c_i. append_operation ("MR", meas_qubits, misassign_err)
    coord = distance
    for m in range(meas_len, 0, -1):
        c_i. append_operation ("DETECTOR", [stim.target_rec(-(m))], (coord, 0))
        coord -= 2
        
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    c = stim.Circuit ()
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )  
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:   
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c. append_operation ("MR", meas_qubits, misassign_err)
    sup = erase_factor*erase_len + meas_len
    coord = distance
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + sup)) ], (coord, 0))
        coord -= 2
    
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    c *= rounds

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:   
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    # BITFLIP DET
    c. append_operation ("MR", data_qubits, misassign_err)
    sup = erase_factor*erase_len + meas_len
    coord = distance
    ct = 0
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + 1)), stim.target_rec(-(m + sup + 1)) ], (coord, 1))
        coord -= 2
        ct += 1

    
    c. append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], (0))

    c = c_i + c

    dem = mwpf.heralded_dem.HeraldedDetectorErrorModel.of(c)

    c = mwpf.heralded_dem.add_herald_detectors(c)


    return {'circuit': c, 'dem': dem}

## Zoomed out logical err vs physical err plot

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-3,-0.9,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=one_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(one_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=10**(4),
        max_errors=10**(4),
        count_detection_events=True,
        save_resume_filepath = "one_det.csv",
        hint_num_tasks = err_list_size*d_list_size,
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat 
    t+=1


# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


Starting 4 workers...
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf <1m       9999       10000 p=0.001,d=3  
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf   ?          0        9999 p=0.001,d=3  
Starting 4 workers...
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf <1m       9727       10000 p=0.001,d=5  
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf <1m       7500       10000 p=0.001,d=5  
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf <1m       3656       10000 p=0.001,d=5  
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf   ?          0       10000 p=0.001,d=5  
Starting 4 workers...
1 tasks left:
  workers decoder eta shots_left errors_left json_metadata
        4    mwpf <1m       9727       10000 p=0.001,d=9  
1 tasks le

## Threshold

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1.7,-1.2,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10
x_factor = 1 + bitflip_factor

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=one_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(one_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



circ_list =  generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_one_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1

thresh_one_det_stats = collected_stats

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: (x_factor)* stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

In [ ]:
err_list_size = 12
d_list_size = 2

err_list = np.logspace(-1.7,-1.2,err_list_size)
d_list = [7,15]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=one_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(one_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=100).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_one_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1

thresh_one_det_stats = collected_stats

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: (x_factor)* stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

## Threshold Plot

In [ ]:
thresh_one_det_stats = sinter.stats_from_csv_files("threshold_one_det.csv")
# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=thresh_one_det_stats,
    x_func=lambda stats: (x_factor)* stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
ax.set_xlim(3e-2, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

# Two Heralded Errors Per Round

## Case 1: Erasure error before CNOTs and before pauli measurement

In [ ]:
def two_det_get_circuit_and_dem(rounds, distance, bit_flip_rate_per_round, heralded_erasure_err, misassign_err, case = 1):

    # initialize circuit info
    meas_len = distance - 1
    all_len = ((2 * distance) - 1)
    erase_len = all_len
    if case == 1:
        erase_factor = 4
    elif case == 2:
        erase_factor = 6
    
    qubits = list(range(all_len))
    meas_qubits = list(qubits [1::2])
    data_qubits = list(qubits[0::2])
    erase_qubits_0 = list(range(0,all_len*2))
    if case == 1:
        erase_qubits_1 = list(range(0,all_len*2))
        erase_qubits_2 = qubits
    elif case == 2:
        erase_qubits_1 = list(range(0,all_len*3))
        erase_qubits_2 = list(range(0,all_len*2))


    cnot1 = qubits [:-1:1]
    cnot2 = qubits [:0:-1]
    
    

    # Build circuit.

    c_i = stim.Circuit()
    c_i. append_operation ("R", qubits)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    ### CNOTS
    c_i. append_operation ("CNOT", cnot1)
    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits_0, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    c_i. append_operation ("CNOT", cnot2)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c_i. append_operation ("MR", meas_qubits, misassign_err)
    coord = distance
    for m in range(meas_len, 0, -1):
        c_i. append_operation ("DETECTOR", [stim.target_rec(-(m))], (coord, 0))
        coord -= 2
        
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    c = stim.Circuit ()
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_1, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )  
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c. append_operation ("MR", meas_qubits, misassign_err)
    sup = erase_factor*erase_len + meas_len
    coord = distance
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + sup)) ], (coord, 0))
        coord -= 2
    
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    c *= rounds

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_1, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    # BITFLIP DET
    c. append_operation ("MR", data_qubits, misassign_err)
    sup = erase_factor*erase_len + meas_len
    coord = distance
    ct = 0
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + 1)), stim.target_rec(-(m + sup + 1)) ], (coord, 1))
        coord -= 2
        ct += 1

    
    c. append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], (0))

    c = c_i + c

    dem = mwpf.heralded_dem.HeraldedDetectorErrorModel.of(c)

    c = mwpf.heralded_dem.add_herald_detectors(c)


    return {'circuit': c, 'dem': dem}

### Zoomed out logical vs physical err rate

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-4,-1,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list


circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_taskstat: List[sinter.TaskStats]= sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        print_progress = True
        )
    collected_stats = collected_stats + collected_taskstat
    t+=1

for i in range(d_list_size*err_list_size):
    collected_stats[i].to_csv_line("threshold_two_det.csv")
    
two_det_stats = collected_stats
# two_det_stats.to_csv('two_det_stats.csv')

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


### Threshold

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1.7,-1.2,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list


circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_two_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1

# for i in range(d_list_size*err_list_size):
#     collected_stats[i].to_csv_line("threshold_two_det.csv")
    
# two_det_stats = collected_stats

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1,-0.6,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list


circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_two_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1

# for i in range(d_list_size*err_list_size):
#     collected_stats[i].to_csv_line("threshold_two_det.csv")
    
# two_det_stats = collected_stats

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

### Threshold Plot

In [ ]:
thresh_two_det_stats = sinter.stats_from_csv_files("threshold_two_det.csv")
# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=thresh_two_det_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
ax.set_ylim(9e-2, 1)
ax.set_xlim(9e-2, 1.2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()

plt.vlines(1.07e-1, 1e-2, 1, "red")
fig.set_dpi(120) 

In [ ]:
thresh_two_det_stats = sinter.stats_from_csv_files("threshold_two_det.csv")
# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=thresh_two_det_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(9e-2, 1)
ax.set_xlim(3e-2, 1.2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()

plt.vlines(1.07e-1, 1e-6, 1, "red")
fig.set_dpi(120) 

## Case 2: Before and after CNOTs

In [ ]:
def two_det_get_circuit_and_dem_2(rounds, distance, bit_flip_rate_per_round, heralded_erasure_err, misassign_err, case = 1):

    # initialize circuit info
    meas_len = distance - 1
    all_len = ((2 * distance) - 1)
    erase_len = all_len
    if case == 1:
        erase_factor = 4
    elif case == 2:
        erase_factor = 6
    
    qubits = list(range(all_len))
    meas_qubits = list(qubits [1::2])
    data_qubits = list(qubits[0::2])
    erase_qubits_0 = qubits
    erase_qubits_1 = list(range(0,all_len*2))
    if case == 1:
        erase_qubits_2 = list(range(0,all_len*2))
    elif case == 2:
        erase_qubits_2 = list(range(0,all_len*3))


    cnot1 = qubits [:-1:1]
    cnot2 = qubits [:0:-1]
    
    

    # Build circuit.

    c_i = stim.Circuit()
    c_i. append_operation ("R", qubits)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits_0, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
        

    ### CNOTS
    c_i. append_operation ("CNOT", cnot1)
    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    c_i. append_operation ("CNOT", cnot2)

    # Err block
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c_i. append_operation ("MR", meas_qubits, misassign_err)
    coord = distance
    for m in range(meas_len, 0, -1):
        c_i. append_operation ("DETECTOR", [stim.target_rec(-(m))], (coord, 0))
        coord -= 2
        
    c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    c = stim.Circuit ()
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_1, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )  
    # HERALD + DET
    c = c +  one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    # BITFLIP DET
    c. append_operation ("MR", meas_qubits, misassign_err)
    sup = 4*erase_len + meas_len
    coord = distance
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + sup)) ], (coord, 0))
        coord -= 2
    
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    c *= rounds

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c + one_time_get_herald_subcircuit(erase_qubits_1, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    # Err block
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD + DET
    c = c +  one_time_get_herald_subcircuit(erase_qubits_2, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)


    # BITFLIP DET
    c. append_operation ("MR", data_qubits, misassign_err)
    sup = 4*erase_len + meas_len
    coord = distance
    ct = 0
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + 1)), stim.target_rec(-(m + sup + 1)) ], (coord, 1))
        coord -= 2
        ct += 1

    
    c. append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], (0))

    c = c_i + c

    dem = mwpf.heralded_dem.HeraldedDetectorErrorModel.of(c)

    c = mwpf.heralded_dem.add_herald_detectors(c)


    return {'circuit': c, 'dem': dem}

### Zoomed out logical vs physical err rate

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-2,-0.6,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem_2(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem_2(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list


circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "two_det_2.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


### Threshold

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1.7,-1.2,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem_2(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem_2(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_two_det_2.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1



# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


In [ ]:
err_list_size = 3
d_list_size = 4

err_list = np.logspace(-1.2,-1,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=two_det_get_circuit_and_dem_2(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(two_det_get_circuit_and_dem_2(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



circ_list = generate_circuits()
collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = circ_list[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_two_det_2.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1



# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


### Threshold plot

In [ ]:
thresh_two_det_stats_2 = sinter.stats_from_csv_files("threshold_two_det_2.csv")
# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=thresh_two_det_stats_2,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(9e-2, 1)
ax.set_xlim(3e-2, 1.2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()

plt.vlines(1.053e-1, 1e-6, 1, "red")
fig.set_dpi(120) 

# Three Heralded Errors per Round

In [ ]:
def three_det_get_circuit_and_dem(rounds, distance, bit_flip_rate_per_round, heralded_erasure_err, misassign_err, case = 1):

    # initialize circuit info
    meas_len = distance - 1
    all_len = ((2 * distance) - 1)
    erase_len = all_len

    if case == 1:
        erase_factor = 4
    elif case == 2:
        erase_factor = 6
    
    qubits = list(range(all_len))
    meas_qubits = list(qubits [1::2])
    data_qubits = list(qubits[0::2])

    cnot1 = qubits [:-1:1]
    cnot2 = qubits [:0:-1]

    if case == 1:
        erase_qubits = qubits
    elif case == 2:
        erase_qubits = list(range(0,all_len*2))
 

    
    

    # Build circuit.

    c_i = stim.Circuit()
    c_i. append_operation ("R", qubits)
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    ### CNOTS
    c_i. append_operation ("CNOT", cnot1)
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c_i. append_operation ("CNOT", cnot2)

    # HERALD DET
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c_i. append_operation ("MR", meas_qubits, misassign_err)
    coord = distance
    for m in range(meas_len, 0, -1):
        c_i. append_operation ("DETECTOR", [stim.target_rec(-(m))], (coord, 0))
        coord -= 2
    
    c_i. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c_i = c_i + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c_i. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
        

    c = stim.Circuit ()
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    # BITFLIP DET
    c. append_operation ("MR", meas_qubits, misassign_err)
    sup = 4*erase_len + meas_len
    coord = distance
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + sup)) ], (coord, 0))
        coord -= 2
    
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    
    c *= rounds

    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    ### CNOTS
    c. append_operation ("CNOT", cnot1)
    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)
    c. append_operation ("CNOT", cnot2)

    c. append_operation ("X_ERROR", qubits , bit_flip_rate_per_round )
    # HERALD DET
    c = c + one_time_get_herald_subcircuit(erase_qubits, qubits, erase_len, heralded_erasure_err)
    if case == 2:
        c. append_operation ("HERALDED_ERASE", qubits, heralded_erasure_err)

    # BITFLIP DET
    c. append_operation ("MR", data_qubits, misassign_err)
    sup = 4*erase_len + meas_len
    coord = distance
    ct = 0
    for m in range(meas_len, 0, -1):
        c. append_operation ("DETECTOR", [stim.target_rec(-(m)), stim.target_rec(-(m + 1)), 
                                          stim.target_rec(-(m + sup + 1)) ], (coord, 1))
        coord -= 2
        ct += 1

    
    c. append("OBSERVABLE_INCLUDE", [stim.target_rec(-1)], (0))

    c = c_i + c

    dem = mwpf.heralded_dem.HeraldedDetectorErrorModel.of(c)

    c = mwpf.heralded_dem.add_herald_detectors(c)


    return {'circuit': c, 'dem': dem}

## Zoomed Out Logical vs physical err rate

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-2,-1,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=three_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(three_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = generate_circuits()[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size
        )
    collected_stats = collected_stats + collected_stat 
    t+=1

three_det_stats = pd.Series(collected_stats)
three_det_stats.to_csv('three_det_stats.csv')

# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


## Threshold

In [ ]:
err_list_size = 6
d_list_size = 4
d_list = [3,5,9,11]

err_list = np.logspace(-1.3,-0.5,err_list_size)
bitflip_factor = 1/10


def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=three_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(three_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = generate_circuits()[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_three_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1



# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 

In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1.7,-1.2,err_list_size)

bitflip_factor = 1/10


def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=three_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(three_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = generate_circuits()[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_three_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1



# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


In [ ]:
err_list_size = 10
d_list_size = 4

err_list = np.logspace(-1,-0.8,err_list_size)
d_list = [3, 5, 9, 11]

bitflip_factor = 1/10

def generate_example_tasks():
    for p in err_list:
        for d in d_list:
            yield sinter.Task(
                circuit=three_det_get_circuit_and_dem(d*3, d,  
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'],
                json_metadata={
                    'p': p,
                    'd': d,
                },
            )

def generate_circuits():
    circ_list = []
    for p in err_list:
        for d in d_list:
            circ_list.append(three_det_get_circuit_and_dem(d*3, d, 
                                            bitflip_factor*p, 
                                            p, 
                                            0)['circuit'])
    return circ_list



collected_stats = list()
t = 0
for task in generate_example_tasks():
    task = [task]
    circ = generate_circuits()[t]
    collected_stat: List[sinter.TaskStats] = sinter.collect(
        num_workers=4,
        tasks=task,
        decoders=['mwpf'],
        custom_decoders = { "mwpf": SinterMWPFDecoder(cluster_node_limit=50).with_circuit(circ) },
        max_shots=int(10**(4)),
        max_errors=int(10**(4)),
        count_detection_events=True,
        hint_num_tasks = err_list_size*d_list_size,
        save_resume_filepath = "threshold_three_det.csv",
        print_progress = True
        )
    collected_stats = collected_stats + collected_stat
    t+=1



# Render a matplotlib plot of the data.
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_stats,
    x_func=lambda stats: stats.json_metadata['p'],
    group_func=lambda stats: stats.json_metadata['d'],
)
# ax.set_ylim(5e-6, 1e-1)
# ax.set_xlim(5e-6, 2e-1)
ax.loglog()
ax.set_title("Repetition Code Error Rates (Herald and Bitflip Errors)")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Shot")
ax.grid(which='major')
ax.grid(which='minor')
ax.legend()
fig.set_dpi(120) 


# Putting Together the plots


In [ ]:

fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
plt.rc('font', size=10) #controls default text size
plt.rc("axes", titlesize=11)
plt.rc("axes", labelsize=17)
plt.rc('xtick', labelsize=7) #fontsize of the x tick labels
plt.rc('ytick', labelsize=7) #fontsize of the y tick labels
plt.rc('legend', fontsize=8)


bitflip_factor = 1/10
x_factor = 1 + bitflip_factor


thresh_one_det_stats = sinter.stats_from_csv_files("threshold_one_det.csv")
thresh_two_det_stats = sinter.stats_from_csv_files("threshold_two_det.csv")
thresh_three_det_stats = sinter.stats_from_csv_files("threshold_three_det.csv")

# Render a matplotlib plot of the data.



# fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax1,
    stats=thresh_one_det_stats,
    x_func=lambda stat: (x_factor)*(stat.json_metadata['p']),
    group_func=lambda stat: stat.json_metadata['d'],
    #failure_units_per_shot_func=lambda stat: stat.json_metadata['r'],
)

sinter.plot_error_rate(
    ax=ax2,
    stats=thresh_two_det_stats,
    x_func=lambda stat: (x_factor)*(stat.json_metadata['p']),
    group_func=lambda stat: stat.json_metadata['d'],
    #failure_units_per_shot_func=lambda stat: stat.json_metadata['r'],
)

sinter.plot_error_rate(
    ax=ax3,
    stats=thresh_three_det_stats,
    x_func=lambda stat: (x_factor)*(stat.json_metadata['p']),
    group_func=lambda stat: stat.json_metadata['d'],
    #failure_units_per_shot_func=lambda stat: stat.json_metadata['r'],
)


plt.xlim(5e-2, 1.7e-1)
fig.supylabel("Logical Error Rate per Shot")
fig.supxlabel("Phyical Error Rate (Erasure and Pauli Checks)")

ax1.loglog()
ax1.set_title("Repetition Code Error Rates")

ax1.grid(which='major')
ax1.set_ylim((1e-1,1))
# ax1.legend()
ax1.vlines(1.2e-1, 1e-2, 1, "red")

ax2.loglog()

ax2.grid(which='major')
ax2.set_ylim((1e-1,1))
#ax2.legend()
ax2.vlines(1.21e-1, 1e-2, 1, "red")

ax3.loglog()
ax3.grid(which='major')
ax3.set_ylim((1e-1,1))
ax3.vlines(1.1e-1, 1e-7, 1, "red")
ax3.legend()

plt.set_dpi(120)
